In [8]:
import pymongo
import requests
import pandas as pd

from os import getenv
from datetime import datetime
from dotenv import load_dotenv

from config import contract_collection, prices_collection
from helper import contract_list


load_dotenv()

import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

In [9]:
def get_collection_activity(metadata, limit, collection=None, collectionssetid=None, community=None, continuation=None):
    url = 'https://api.reservoir.tools/'
    end_point = 'collections/activity/v5'
    headers = {'accept': '*/*', 'x-api-key': f"{getenv('RESERVOIR')}"}
    if community == None:
        r = requests.get(url + end_point, params=f'?sortBy=eventTimestamp&includeMetadata={metadata}&limit={limit}', headers=headers)
    else:
        r = requests.get(url + end_point, params=f'?sortBy=eventTimestamp&includeMetadata={metadata}&limit={limit}&community={community}', headers=headers)
    data = r.json()
    return data

In [10]:
contract_db = contract_list()

test = get_collection_activity(True, 20, community='artblocks')
new_bids = []
for item in test['activities']:
    if item['type'] == 'bid':
        if item['order']['criteria']['kind'] == 'collection':
            actions = {}
            actions['contract'] = item['contract']
            actions['collection_name'] = item['collection']['collectionName']
            actions['collection_id'] = item['order']['criteria']['data']['collection']['id']
            if item['order']['criteria']['data']['collection']['id'] not in contract_db:
                new_bids.append(actions)

if new_bids:               
    contract_collection().insert_many(new_bids)

contract_db = contract_list()
df = pd.DataFrame(contract_db)

In [11]:
def get_collection_info(limit, topbid, normalize, c_id=None):
    url = 'https://api.reservoir.tools/'
    end_point = 'collections/v5'
    headers = {'accept': '*/*', 'x-api-key': f"{getenv('RESERVOIR')}"}
    r = requests.get(url + end_point, params=f'id={c_id}&includeTopBid={topbid}&normalizeRoyalties={normalize}&sortBy=allTimeVolume&limit={limit}', headers=headers)
    data = r.json()
    return data

In [20]:
new_bids = []
for c in contract_db:
    collections = get_collection_info(1, True, False, c_id=c)
    for collection in collections['collections']:
        project = {}
        project['Collection'] = collection['name']
        project['7day Volume'] = round(collection['volume']['7day'], 2)
        project['Lowest Floor Price'] = f"{str(collection['floorAsk']['sourceDomain'])} : {round(collection['floorAsk']['price']['amount']['decimal'], 3)} {collection['floorAsk']['price']['currency']['symbol']}"
        try:
            project['Highest Collection Offer'] = f"{str(collection['topBid']['sourceDomain'])} : {round(collection['topBid']['price']['amount']['decimal'], 3)} {collection['topBid']['price']['currency']['symbol']}"
            project['Spread'] = f"{round(((collection['floorAsk']['price']['amount']['decimal'] - collection['topBid']['price']['amount']['decimal']) / collection['floorAsk']['price']['amount']['decimal']) * 100, 2)}%"
        except TypeError:
            project['Highest Collection Offer'] = 'None'
            project['Spread'] = 'None'
        new_bids.append(project)
    

df = pd.DataFrame(new_bids)    

In [21]:
df

,Collection,7day Volume,Lowest Floor Price,Highest Collection Offer,Spread
0,Low Tide by Artem Verkhovskiy x Andy Shaw,0.00,opensea.io : 0.199 ETH,opensea.io : 0.07 WETH,64.82%
1,CryptoGodKing by Steve Pikelny,1.21,sansa.xyz : 0.8 ETH,opensea.io : 0.4 WETH,50.0%
2,celestial cyclones by hideo,1.18,opensea.io : 0.3 ETH,opensea.io : 0.111 WETH,63.0%
3,Antennas by gcrll,0.37,opensea.io : 0.25 ETH,opensea.io : 0.1 WETH,60.0%
4,Transitions by Jason Ting x Matt Bilfield,3.00,opensea.io : 0.08 ETH,opensea.io : 0.057 WETH,28.44%
...,...,...,...,...,...
168,LeWitt Generator Generator by Mitchell F. Chan,1.65,opensea.io : 0.69 ETH,opensea.io : 0.33 WETH,52.17%
169,Paths by Darien Brito,0.63,opensea.io : 0.24 ETH,opensea.io : 0.13 WETH,45.83%
170,Quantum Collapses by Insigħt,0.00,opensea.io : 0.369 ETH,opensea.io : 0.16 WETH,56.64%
171,InC by hex6c,0.00,opensea.io : 0.18 ETH,opensea.io : 0.02 WETH,88.89%


In [13]:
collections

{'collections': [{'id': '0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270:294000000:294999999',
   'slug': 'alien-dna-by-shvembldr',
   'createdAt': '2022-04-05T17:44:25.130Z',
   'name': 'Alien DNA by Shvembldr',
   'image': 'https://media.artblocks.io/294000000.png',
   'banner': None,
   'discordUrl': None,
   'externalUrl': 'https://www.shvembldr.com',
   'twitterUsername': None,
   'openseaVerificationStatus': None,
   'description': 'In the year 3075 of the Second Nuclear Age, a laboratory of an unknown race of aliens was discovered on the planet Vtolmr while drilling a well of molecular oil. Its presumed purpose is to study genetic mutations of races from neighboring planets. During the laboratory investigation, 512 silicon impressions with DNA of different alien species were extracted. Modern methods of quantum analysis do not allow the deciphering of the data obtained. At this point, their value is purely aesthetic. But that may change in time.\n\nPress 1 to 9 - change the pixel den

In [ ]:
df[~df['Spread'].isin(['None'])].sort_values(by='Collection', ascending=True).to_csv('spread.csv', index=False)

In [ ]:
prices_db = prices_collection()

In [ ]:
pd.DataFrame(list(prices_db.find().sort('7day Volume', pymongo.DESCENDING))).drop('_id', axis=1)

In [ ]:
get_activity(False, 20, 'desc')

In [ ]:
url = 'https://api.reservoir.tools/'

response = requests.get(url, json={'limit': 20, 'includeMetadata': False}, headers=headers)

print(response.url)
test = response.json()

In [ ]:
url = "https://api.reservoir.tools/orders/bids/v5?community=artblocks&includeCriteriaMetadata=false&includeRawData=false&normalizeRoyalties=false&sortBy=createdAt&limit=50"
headers = {
    "accept": "*/*",
    "x-api-key": "demo-api-key"
}

In [ ]:
def mongo_collection():
    client = pymongo.MongoClient(getenv('MONGO'))
    db = client.nonfungible 
    return db.globalcontracts

In [ ]:
db = mongo_collection()

db.find_one()

In [ ]:
start_block = init_collection.find({'blockchain': 'flow'}).sort('blockNumber', -1).limit(1).next()['blockNumber'] - 25